In [ ]:
# tag::e2e_imports[]
import h5py

from keras.models import Sequential
from keras.layers import Dense

from dlgo.agent.predict import DeepLearningAgent, load_prediction_agent
from dlgo.data.parallel_processor import GoDataProcessor
from dlgo.encoders.sevenplane import SevenPlaneEncoder
# end::e2e_imports[]

# tag::e2e_processor[]
go_board_rows, go_board_cols = 19, 19
nb_classes = go_board_rows * go_board_cols
encoder = SevenPlaneEncoder((go_board_rows, go_board_cols))
processor = GoDataProcessor(encoder=encoder.name())

num_games = 10000
X, y = processor.load_go_data('train', num_games)
validationX, validationY = processor.load_go_data('test', num_games)
print('X.shape', X.shape)
print('y.shape', y.shape)
print('validationX.shape', validationX.shape)
print('validationY.shape', validationY.shape)
# end::e2e_processor[]

>>> Reading cached index page
KGS-2019_04-19-1255-.tar.gz 1255
KGS-2019_03-19-1478-.tar.gz 1478
KGS-2019_02-19-1412-.tar.gz 1412
KGS-2019_01-19-2095-.tar.gz 2095
KGS-2018_12-19-1992-.tar.gz 1992
KGS-2018_11-19-1879-.tar.gz 1879
KGS-2018_10-19-1209-.tar.gz 1209
KGS-2018_09-19-1587-.tar.gz 1587
KGS-2018_08-19-1447-.tar.gz 1447
KGS-2018_07-19-949-.tar.gz 949
KGS-2018_06-19-1002-.tar.gz 1002
KGS-2018_05-19-1590-.tar.gz 1590
KGS-2018_04-19-1612-.tar.gz 1612
KGS-2018_03-19-833-.tar.gz 833
KGS-2018_02-19-1167-.tar.gz 1167
KGS-2018_01-19-1526-.tar.gz 1526
KGS-2017_12-19-1488-.tar.gz 1488
KGS-2017_11-19-945-.tar.gz 945
KGS-2017_10-19-1351-.tar.gz 1351
KGS-2017_09-19-1353-.tar.gz 1353
KGS-2017_08-19-2205-.tar.gz 2205
KGS-2017_07-19-1191-.tar.gz 1191
KGS-2017_06-19-910-.tar.gz 910
KGS-2017_05-19-847-.tar.gz 847
KGS-2017_04-19-913-.tar.gz 913
KGS-2017_03-19-717-.tar.gz 717
KGS-2017_02-19-525-.tar.gz 525
KGS-2017_01-19-733-.tar.gz 733
KGS-2016_12-19-1208-.tar.gz 1208
KGS-2016_11-19-980-.tar.gz 980


AssertionError: 

In [ ]:
#from dlgo.httpfrontend import get_web_app
from dlgo.networks import large
from keras.callbacks import ModelCheckpoint  # <1>
# <1> With model checkpoints we can store progress for time-consuming experiments
from keras.optimizers import Adagrad

#  tag::e2e_model[]
input_shape = (encoder.num_planes, go_board_rows, go_board_cols)
model = Sequential()
network_layers = large.layers(input_shape)
for layer in network_layers:
    model.add(layer)
model.add(Dense(nb_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adagrad(), metrics=['accuracy'])
print(model.summary())

epochs = 300
batch_size = 256
model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_data=(validationX, validationY), verbose=1,
          callbacks=[ModelCheckpoint('../checkpoints/small_model_epoch_{epoch}.keras')])
# <5> After each epoch we persist a checkpoint of the model.
model.evaluate(validationX, validationY, batch_size=batch_size)  
# end::e2e_model[]

# tag::e2e_agent[]
deep_learning_bot = DeepLearningAgent(model, encoder)
deep_learning_bot.serialize(h5py.File("../agents/deep_bot.h5", "w"))
# end::e2e_agent[]

In [ ]:
# tag::e2e_load_agent[]
model_file = h5py.File("../agents/deep_bot.h5", "r")
bot_from_file = load_prediction_agent(model_file)

web_app = get_web_app({'predict': bot_from_file})
web_app.run()
# end::e2e_load_agent[]